https://colab.research.google.com/drive/1Q9xOhs_1Jz6g-FLaemPR5yHhTXpzKx1u#scrollTo=ExxUgPNEK50v

In [2]:
#Final
import mysql.connector as SQLC
class DB:
#database connection and extraction of sequences and id_genebanks
    def __init__(self, password):        
        if password != "20221207":
            raise Exception("The password is wrong!")

        else:
            self.cnx = SQLC.connect(
                    host ="geo.di.uminho.pt",
                    user ="bioinformatica",
                    password = password, #"20221207"
                    database ="AP_db_KRG")
            self.sequences = []
        
    def extract_sequences(self):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT sequence FROM Gene")
        self.sequences = cursor.fetchall()
        cursor.close()
        
    def extract_ids(self):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT ID_genebank FROM Gene")
        self.sequences = cursor.fetchall()
        cursor.close()
    
    def __str__(self): 
        return self.sequences

In [2]:
DB("2022")

Exception: The password is wrong!

In [3]:
#Final
import unittest
class TestDB(unittest.TestCase):
#testing DB class
    def test_setUp(self):
        self.assertTrue(DB("20221207"))
        with self.assertRaises(Exception) as error:
            self.db1 = DB("2022120")
            self.db2 = DB("20221208")
            self.db3 = DB("")
        self.assertEqual(error.exception.args[0], "The password is wrong!")

    def test_extract_sequences(self):
        self.db = DB("20221207")
        self.db.extract_sequences()
        self.assertIsNotNone(self.db.sequences)
        
    def test_extract_ids(self):
        self.db = DB("20221207")
        self.db.extract_ids()
        self.assertIsNotNone(self.db.sequences)
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestDB)
unittest.TextTestRunner(verbosity=3).run(suite)   

test_extract_ids (__main__.TestDB) ... ok
test_extract_sequences (__main__.TestDB) ... ok
test_setUp (__main__.TestDB) ... ok

----------------------------------------------------------------------
Ran 3 tests in 3.320s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [4]:
#FINAL
class SimpleBlast:
    def __init__(self, w, seq):
        self.w = w
        self.seq = seq
        
    def query_map(self, query):
    #returns a dictionary where key is query of length w and values are 
    #the indexes of query that match with key
        
        query_map = {}
        for i in range(len(query) - self.w + 1):
            window = query[i: i + self.w]
            if window in query_map:
                query_map[window].append(i)
            else:
                query_map[window] = [i]
        return query_map 

    def hits(self, query_map, sequence):
    #returns list of tuples where 1st index is from query and 2nd is from seq
        hits = []
        for window_query, query_offsets in query_map.items():
            if window_query in sequence:
                seq_offset = sequence.find(window_query)
                while seq_offset != -1: #because of find()
                    for query_offset in query_offsets:
                        hits.append((query_offset, seq_offset))
                    seq_offset = sequence.find(window_query, seq_offset + 1) #because while loop and find()
        return sorted(hits)

    def extend_hit_dir(self, query, sequence, q_offset, s_offset, direction):
    #direction: -1 or +1
        count = 0
        matches = 0
        while q_offset >= 0 and s_offset >= 0 and q_offset < len(query) and s_offset < len(sequence):
            count += 1
            if query[q_offset] == sequence[s_offset]:
                matches += 1
            if 2 * matches < count:
                break
            q_offset += direction
            s_offset += direction
        return q_offset - direction, s_offset - direction, matches, count

    def extend_hit(self, query, seq, hit, w):
    #given a hit, will extend the search of matches to left and right    
        o1, o2 = hit
        left  = self.extend_hit_dir(query, seq, o1 - 1, o2 - 1, -1)
        right = self.extend_hit_dir(query, seq, o1 + w, o2 + w, +1)

        O1, O2, ML, SL = left
        _,   _, MR, SR = right

#         print({'ML':ML, 'SR':SR, 'SL':SL,'MR':MR})
        return O1, O2, w + SL + SR, ML + w + MR
    
    def best_hit(self, query, seq, w):
    #finds best hit from all hits and respective extensions
        query_map = self.query_map(query)
        hits = self.hits(query_map, seq)
        best_hit = None
        best_score = -1
        for hit in hits:
            o1, o2, l, m = self.extend_hit(query, seq, hit, w)
            score = m - (l - m)
            if score > best_score or (score == best_score and l < best_hit[2]):
                best_hit = (o1, o2, l, m)
                best_score = score
        return best_hit
    
# best_hit = (A,B,C,D)
# A = start position of the best match in the query
# B = starting position of the best match in the sequence
# C = best match size
# D = number of matches between nucleotides of the best match

In [5]:
#Final
class TestSBlast(unittest.TestCase):
    w = 3
    seq = 'AATATGTTATATAATAATATTT'
    query = 'AATATAT'
    result_qm = {'AAT': [0], 'ATA': [1, 3], 'TAT': [2, 4]}
    def setUp(self):
        self.sb = SimpleBlast(self.w, self.seq)
      
    def test_querymap(self):
        self.assertEqual(self.sb.query_map(self.query),self.result_qm)
        
    def test_hits(self):
        result_hit = [(0, 0), (0, 12), (0, 15), (1, 1), (1, 8), (1, 10), (1, 13), (1, 16),
                    (3, 1), (3, 8), (3, 10), (3, 13), (3, 16), (2, 2), (2, 7), (2, 9),
                    (2, 17), (4, 2), (4, 7), (4, 9), (4, 17)]
        self.assertEqual(set(self.sb.hits(self.result_qm, self.seq)), set(result_hit))
        
    def test_extend_hit_dir(self):
        self.assertEqual(self.sb.extend_hit_dir(self.query, self.seq, 1, 16, -1), (0, 15, 2, 2))
        self.assertEqual(self.sb.extend_hit_dir(self.query, self.seq, 1, 16, 1), (6, 21, 5, 6)) 
        
    def test_extend_hit(self):
        self.assertEqual(self.sb.extend_hit(self.query, self.seq, (1,16), self.w), (0, 15, 7, 6))
    
    def test_best_hit(self):
        self.assertEqual(self.sb.best_hit(self.query, self.seq, self.w), (0, 0, 7, 6))  

suite = unittest.TestLoader().loadTestsFromTestCase(TestSBlast)
unittest.TextTestRunner( verbosity=3 ).run( suite )

test_best_hit (__main__.TestSBlast) ... ok
test_extend_hit (__main__.TestSBlast) ... ok
test_extend_hit_dir (__main__.TestSBlast) ... ok
test_hits (__main__.TestSBlast) ... ok
test_querymap (__main__.TestSBlast) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.006s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [6]:
#final
class BlastResults:
    def __init__(self, query, w, password):
        self.query = query
        self.w = w
        self.password = password
        self.sequences_obj = DB(password)
        self.ids_obj = DB(password)
        self.sequences_obj.extract_sequences()
        self.ids_obj.extract_ids()
  
    def process_results(self):
    #joins the best_hit to respective id_genebank (from database)
        all_seqs = []
        for num, seq in enumerate(self.sequences_obj.sequences):
            seq_id = []
            seq = ''.join(seq)
            blast = SimpleBlast(self.w, seq)
            results = blast.best_hit(self.query, seq, self.w)
            seq_id.append(results)
            seq_id.append(''.join(self.ids_obj.sequences[num]))
            all_seqs.append(seq_id)
        return all_seqs
    
    def sort_results(self):
        return sorted(self.process_results(), key=lambda item: item[0][-1] if item[0] else 0, reverse=True)

    
    
    def printing(self):
        # organized printing
        print(
            '{:>9} {:>15} {:>18} {:>12}  {:>5}'.format('Query position', 'Seq position', 'Length of match', 'Max match',
                                                       'Id'))
        for i in self.sort_results():
            if i[0]:
#             print(i)
                print('{:>6} {:>18} {:>15} {:>16}      |'.format(*i[0]), i[1])
        return '_______________________________________________________________'

In [7]:
#Final
import unittest

class TestBlastResults(unittest.TestCase):
#specific results are not being tested because the database can change.     
    def setUp(self):
        self.query = "ACGT"
        self.w = 3
        self.password = "20221207"
        self.blast_results = BlastResults(self.query, self.w, self.password)

    def test_process_results(self):
        all_seqs = self.blast_results.process_results()
        self.assertIsNotNone(all_seqs)
        self.assertIsInstance(all_seqs, list)
        self.assertGreater(len(all_seqs), 0)
        self.assertIsInstance(all_seqs[0], list)
        self.assertEqual(len(all_seqs[0]), 2)

    def test_sort_results(self):
        sorted_results = self.blast_results.sort_results()
        self.assertIsNotNone(sorted_results)
        self.assertIsInstance(sorted_results, list)
        self.assertGreater(len(sorted_results), 0)
        self.assertIsInstance(sorted_results[0], list)
        self.assertEqual(len(sorted_results[0]), 2)
        self.assertTrue(sorted_results == sorted(sorted_results, key=lambda x: x[0][-1] if x[0] else 0, reverse=True))
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestBlastResults)
unittest.TextTestRunner( verbosity=3 ).run( suite )  


test_process_results (__main__.TestBlastResults) ... ok
test_sort_results (__main__.TestBlastResults) ... ok

----------------------------------------------------------------------
Ran 2 tests in 1.534s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [12]:
query = "AUG"
w = 1

# try:
blast_results = BlastResults(query, w, "20221207")

blast_results.printing()


Query position    Seq position    Length of match    Max match     Id
     0                  1               2                1      | A00829.1
     2                  0               1                1      | AAADK0043071.1
     0                  1               2                1      | AAADS0003348.1
     0                 20               1                1      | AAADW0009410.1
     2                  0               1                1      | AAADX0043451.1
     0                  2               2                1      | AAVX01203288.1
     0                 19               1                1      | ABAAM0346030.1
     0                  2               2                1      | BAAF03073813.1
     0                  4               2                1      | DH405380.1
     0                 15               2                1      | EC923692.1
     0                  6               2                1      | EC966285.1
     2                  0               1                

'_______________________________________________________________'

In [10]:
#Final
def validate_inputs(query, window, password):
    if not query.isalpha():
        return "Error: query must be only letters of nucleotides"
    if not window.isdigit():
        return "Error: window must be only numbers"
    if int(window) > len(query):
        return "Error: window must be equal or less than the length of query"
    if password != "20221207":
        return "Error: wrong password"
    return "Inputs are valid."

while True:
    query = input("Enter query: ")
    window = input("Enter window: ")
    password = input("Enter password: ")
    
    result = validate_inputs(query, window, password)
    if result == "Inputs are valid.":
        q = str(query).upper()
        w = int(window)
        break
    else:
        print(result)

blast_results = BlastResults(q, w, password)
blast_results.printing()

Enter query: 12
Enter window: 2
Enter password: 20221207
Error: query must be only letters of nucleotides
Enter query: ã
Enter window: 1
Enter password: 20221207
Query position    Seq position    Length of match    Max match     Id


'_______________________________________________________________'

In [11]:
#Final
class TestValidateInputs(unittest.TestCase):
    def test_valid_inputs(self):
        query = "query"
        window = "3"
        password = "20221207"
        result = validate_inputs(query, window, password)
        self.assertEqual(result, "Inputs are valid.")
        
    def test_invalid_query(self):
        query = "1234"
        window = "3"
        password = "20221207"
        result = validate_inputs(query, window, password)
        self.assertEqual(result, "Error: query must be only letters of nucleotides")
        
    def test_invalid_window(self):
        query = "query"
        window = "abc"
        password = "20221207"
        result = validate_inputs(query, window, password)
        self.assertEqual(result, "Error: window must be only numbers")
        
    def test_invalid_password(self):
        query = "query"
        window = "3"
        password = "12345678"
        result = validate_inputs(query, window, password)
        self.assertEqual(result, "Error: wrong password")
        
    def test_window_larger_than_query(self):
        query = "query"
        window = "10"
        password = "20221207"
        result = validate_inputs(query, window, password)
        self.assertEqual(result, "Error: window must be equal or less than the length of query")
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestValidateInputs)
unittest.TextTestRunner( verbosity=3 ).run( suite )

test_invalid_password (__main__.TestValidateInputs) ... ok
test_invalid_query (__main__.TestValidateInputs) ... ok
test_invalid_window (__main__.TestValidateInputs) ... ok
test_valid_inputs (__main__.TestValidateInputs) ... ok
test_window_larger_than_query (__main__.TestValidateInputs) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.007s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>